In [1]:
import sqlite3
import pandas as pd
import numpy as np
import global_imports
from src import xbox_spreadsheet, origin_parse, report, write_to_excel
import re
import json

# Not necessary in a py file as jupyter does not set the path correctly
import os
if os.getcwd().split('\\')[-1] != global_imports.root_folder:
    os.chdir(global_imports.root_folder)

In [2]:
def import_database_from_sql(sql_query,db_path):
    connector = sqlite3.connect(db_path)
    db = pd.read_sql(sql_query, connector)
    return db

game_piece_type_query = 'SELECT * FROM GamePieceTypes'
game_piece_type = import_database_from_sql(game_piece_type_query, global_imports.main_db_path)

main_db_query = """SELECT GamePieces.releaseKey, GamePieces.gamePieceTypeId, GamePieces.value
    FROM GameLinks
	JOIN GamePieces ON GameLinks.releaseKey = GamePieces.releaseKey"""
main_db = import_database_from_sql(main_db_query, global_imports.main_db_path)

In [3]:
#Add any metadata needed to this list. Valid values are type names
# from GamePieceTypes
column_names = ['title', 'meta']

def split_metadata_into_columns(column_list, db):
    """Splits metadata of the main_db into different rows. Arguments are a list of column_names
    and the dataframe"""

# Creates columns for each item in column_list
    for val in column_list:
        type_id = int(game_piece_type.loc[game_piece_type['type'] == val, 'id'])
        db[val] = db.loc[db['gamePieceTypeId'] == type_id, 'value']

# Drop rows that are null in all of the generated columns. This will remove every row that
# is not in column_list
    db = db.dropna(subset=column_list, how='all')

# Drop every other row we do not need anymore
    db = db.drop(['value','gamePieceTypeId'], axis=1)

# Group by releaseKey since the data is in its own rows. Fill row with NAN if every row is NAN
    db = db.groupby('releaseKey', as_index=True).agg(lambda x: np.nan if x.isnull().all()
    else x.dropna()).reset_index()
    db = db.set_index(np.arange(0, len(db)))
    return db


main_db = split_metadata_into_columns(column_names, main_db)

In [4]:
def extract_date(db):
    """Extract Release Date from metadata and convert it into MM YYYY format"""

    date_pattern = 'releaseDate\":(\d{9,10})'
    def format_date(x):
        date = re.search(date_pattern, x)
        if date:
            val = pd.to_datetime(date.group(1), unit='s')
            val = val.strftime('%Y %b')
            return val
        else:
            return 'No Date'

    db['date'] = db['meta'].apply(format_date)
    db = db.drop('meta', axis=1)
    return db

main_db = extract_date(main_db)

In [5]:
def create_platform(db):
    with open(global_imports.platforms_json) as platform_file:
        platform = json.load(platform_file)

    # Create a regex pattern of all platforms to match and then return the actual platform name
    platform_keys = list(platform.keys())
    platform_pattern = re.compile(r"(\b{}\b)".format("|".join(platform_keys)))

    def platform_extract(x):
        m = platform_pattern.match(x)
        if m:
            return platform[m.group(1)]

    db['platform'] = db['releaseKey'].apply(platform_extract)
    return db

main_db = create_platform(main_db)

In [6]:
def remove_hidden_games_by_gog(db):
    hidden_db_query = 'SELECT releaseKey, isDlc, isVisibleInLibrary FROM ReleaseProperties'
    hidden_db = import_database_from_sql(hidden_db_query, global_imports.main_db_path)

    # Delete everything marked as DLC or not visible in library
    db = db[~db['releaseKey'].isin(hidden_db.loc[hidden_db['isVisibleInLibrary'] == 0, 'releaseKey'])]
    db = db[~db['releaseKey'].isin(hidden_db.loc[hidden_db['isDlc'] == 1, 'releaseKey'])]
    return db

main_db = remove_hidden_games_by_gog(main_db)

In [7]:
def remove_manual_hidden_games_by_user(db):
    user_hidden_db_query = 'SELECT releaseKey, isHidden FROM UserReleaseProperties'
    user_hidden_db = import_database_from_sql(user_hidden_db_query, global_imports.main_db_path)

    # Remove games in main_db but not in user_hidden_db as these were never owned
    db = db.drop(db[~db['releaseKey'].isin(user_hidden_db['releaseKey'])].index)
    db = db[~db['releaseKey'].isin(user_hidden_db.loc[user_hidden_db['isHidden'] == 1, 'releaseKey'])]
    db = db.reset_index(drop=True)
    return db

main_db = remove_manual_hidden_games_by_user(main_db)

In [8]:
# Remove brackets and quotes around titlee
main_db['title'] = main_db['title'].apply(lambda x: x[10:-2])

def format_titles(db, column):

    """Format Title by removing special and unicode characters like trademark"""

    # Remove Trademark and Copyright Symbols
    db[column] = db[column].str.replace('\u2122|\u00AE','')

    # Remove Windows, Windows 10 and any word before it. for Windows 10, - Windows 10 would be removed
    db[column] = db[column].str.replace('\S+ Windows(?: 10)?', '')

    # Remove special apostrophe
    db[column] = db[column].str.replace('’', '')

    # Make any game with The in the middle of the sentence lower case
    db[column] = db[column].str.replace('\sthe\s', ' the ', flags=re.IGNORECASE)

    # Remove the at the beginning of the word
    db[column] = db[column].str.replace('^The\s', '')

    # Make any game with at in the middle of the sentence lower case
    db[column] = db[column].str.replace('\sat\s', ' at ', flags=re.IGNORECASE)
    db[column] = db[column].str.strip()

    return db

main_db = format_titles(main_db, 'title')

In [9]:
def remove_duplicates(db):

    # Delete rows with same title and platform
    db = db.drop_duplicates(subset=['title','platform'], keep='last').reset_index(drop=True)

    # Delete rows with same title in 'Other' Platform
    dup = db[db.duplicated(subset=['title'], keep=False)].sort_values('title')
    db = db.drop(dup[dup['platform'].str.match('Other')].index).reset_index(drop=True)
    db = db[~db['title'].str.contains('trial$',flags=re.IGNORECASE)]
    return db
main_db = remove_duplicates(main_db)

In [10]:
def create_tag(db):

    # Import tags database
    tag_db_query = 'SELECT releaseKey, tag FROM UserReleaseTags'
    tag_db = import_database_from_sql(tag_db_query, global_imports.main_db_path)
    temp = db.merge(tag_db, how='left',left_on=['releaseKey'],right_on=['releaseKey'])

    def create_status(x):
        pattern = re.compile(r"S - (\w+)")
        if pattern.match(x):
            return pattern.match(x).group(1)
        return np.nan

    def create_length(x):
        pattern = re.compile(r"L - (\w+)")
        if pattern.match(x):
            return pattern.match(x).group(1)
        return np.nan

    # Create Status and Length as two separate Columns
    temp['tag'] = temp['tag'].fillna('No tag')
    temp['Status'] = temp['tag'].apply(create_status)
    temp['Length'] = temp['tag'].apply(create_length)
    temp.drop(['tag'], axis=1, inplace=True)
    temp = temp.groupby('releaseKey')[['Status','Length']].first().reset_index()
    db = db.merge(temp, how='left', on='releaseKey')
    return db

# Add the Status and Length to the main Table
main_db = create_tag(main_db)

In [12]:
def format_xbox(db):
    db = pd.DataFrame(db[0], columns=db[1]).reset_index(drop=True)

    # Remove the header and make the first row as header
    new_header = db.iloc[0]
    db = db[1:]
    db.columns = new_header
    db.reset_index(drop=True, inplace=True)

    # Remove Xbox Games and unnecessary columns from the list
    db = db.drop(['Metacritic', 'Genre (Giantbomb)', 'Completion', 'Age', 'Release', 'Months'], axis=1).reset_index(
        drop=True)
    db = db[~(db['System'] == 'Xbox One')]
    db = db.drop(['System'], axis=1).reset_index(drop=True)
    return db

xbox_db = format_xbox(xbox_spreadsheet.import_xbox_gsheet())
xbox_db = format_titles(xbox_db, 'Game')

In [13]:
def remove_xboxgamepass(db, xdb):
    temp = db.loc[db['platform'] == 'Xbox Gamepass'].copy()
    xtemp = xdb.loc[xdb['Status'].str.match(r'Active|Leaving Soon')].copy()

    # Temporarily remove special characters and convert to lowercase for comparison
    def temp_format_xbox(x):
        x = x.lower()
        x = re.sub('\s\(.*\)', '', x)
        x = re.sub('\'', '', x)
        x = re.sub(': (\w+\sedition)', '', x)
        x = re.sub(': (\w+\scut)', '', x)
        x = re.sub(' iii', '3', x)
        x = re.sub(' ii', '2', x)
        x = re.sub(' iv', '4', x)
        x = re.sub(' ix', '9', x)
        x = re.sub(' xv', '15', x)
        x = re.sub('[^A-Za-z0-9]+', '', x)
        return x

    temp.loc[:, 'title'] = temp['title'].apply(temp_format_xbox)
    xtemp.loc[:, 'Game'] = xtemp['Game'].apply(temp_format_xbox)

    # Fetch index of all the games in main but not in Xbox Gamepass list and delete them
    out = temp[~temp['title'].isin(xtemp['Game'])].index
    db.drop(out, inplace=True)
    db.reset_index(drop=True, inplace=True)
    return db

main_db = remove_xboxgamepass(main_db, xbox_db)

In [14]:
def import_origin():
    basic, premiere = origin_parse.origin_games()
    data = pd.DataFrame(basic, columns=['Title', 'Subscription'])
    data = data.append(pd.DataFrame(premiere, columns=data.columns))
    data = data.sort_values('Title').reset_index(drop=True)
    return data

origin_db = import_origin()

In [15]:
main_db = main_db.rename(columns={"title": "Title", "date": "Release", "platform": "Platform"})
xbox_db = xbox_db.rename(columns={"Game": "Title"})
main_db.reset_index(drop = True, inplace = True)
xbox_db.reset_index(drop = True, inplace = True)

In [16]:
def group_platform(db):
    temp = db.groupby('Title').agg({'Platform':'; '.join})
    db = db.merge(temp, how='inner',left_on=['Title'],right_on=['Title'])
    db = db.drop('Platform_x', axis=1)
    db = db.drop_duplicates(subset=['Title', 'Platform_y'], keep='last')
    db = db.reset_index(drop=True)
    db = db.rename(columns={"Platform_y": "Platform"})
    return db

main_db = group_platform(main_db)

In [17]:
def manual_tagging(db):
    with open('data/tag.txt') as f:
        row = f.read().split('\n')
        for i in row:
            col = i.split(',')
            db.loc[db['Title'].str.contains(col[0]), 'Length'] = col[1]
            db.loc[db['Title'].str.contains(col[0]), 'Status'] = col[2]
    return db

main_db = manual_tagging(main_db).reset_index(drop=True)

# Sort values by Title ignoring case
main_db['Upper'] = main_db['Title'].str.upper()
main_db.sort_values(by='Upper', inplace=True)
del main_db['Upper']

In [18]:
def add_gameplay_time(db):
    time_query = 'SELECT ProductName, StatTotalFullRuntime FROM Applications'
    time_db = import_database_from_sql(time_query, global_imports.gtt_db_path)
    db = db.merge(time_db, how='left', left_on='Title', right_on='ProductName')
    db = db.drop('ProductName', axis=1)
    db['StatTotalFullRuntime'] = db['StatTotalFullRuntime'].fillna(0)
    db = db.rename(columns={"StatTotalFullRuntime": "GameplayTime"})

    def convert_to_hours(x):
        if x:

            # The value of gameplay has additional zeroes. Remove zeroes and convert to minutes
            x = x/(10000000*60)
            hours = int(x/60)
            minutes = int(x%60)

            # Return output according to whether minutes or hours are zero
            if hours and minutes:
                return str(hours) + ' Hours ' + str(minutes) + ' Minutes'
            elif hours:
                return str(hours) + ' Hours'
            elif minutes:
                return str(minutes) + ' Minutes'
        else:
            return 0
    db['GameplayTime'] = db['GameplayTime'].apply(convert_to_hours)
    return db

main_db = add_gameplay_time(main_db)

In [19]:
db_list = {'main_db':main_db, 'xbox_db':xbox_db, 'origin_db':origin_db}

def save_dataframe(dict_of_db):
    for key,value in dict_of_db.items():
        old = 'data/' + str(key) + '_old'
        new = 'data/' + str(key)
        os.remove(old)
        os.rename(new, old)
        value.to_pickle(new)

save_dataframe(db_list)

In [20]:
write_to_excel.main()

FileCreateError: [Errno 13] Permission denied: 'Games.xlsx'

In [ ]:
report.main()

In [ ]:
temp1 = 'SELECT releaseKey, isDlc, isVisibleInLibrary FROM ReleaseProperties'
temp2 = import_database_from_sql(temp1, global_imports.main_db_path)

temp1 = 'SELECT releaseKey, isHidden FROM UserReleaseProperties'
temp3 = import_database_from_sql(temp1, global_imports.main_db_path)



In [ ]:
temp3[temp3['releaseKey'] == 'origin_Origin.OFR.50.0002732']
main_db[main_db['Title'].str.contains('Battlefield')]

